## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sampit,ID,-2.5333,112.9500,76.46,95,100,1.63,light rain
1,1,Palauig,PH,15.4350,119.9045,80.65,70,48,1.34,scattered clouds
2,2,Medea,DZ,36.2642,2.7539,55.40,88,40,2.30,scattered clouds
3,3,Palmer,US,42.1584,-72.3287,35.60,35,20,8.05,few clouds
4,4,Chokurdakh,RU,70.6333,147.9167,-22.81,86,38,3.33,scattered clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sampit,ID,-2.5333,112.9500,76.46,95,100,1.63,light rain
1,1,Palauig,PH,15.4350,119.9045,80.65,70,48,1.34,scattered clouds
5,5,Rikitea,PF,-23.1203,-134.9692,78.67,69,100,9.84,overcast clouds
10,10,Itaituba,BR,-4.2761,-55.9836,86.00,74,0,1.81,clear sky
13,13,Kahului,US,20.8947,-156.4700,77.00,50,20,11.50,few clouds
14,14,Prieska,ZA,-29.6641,22.7474,79.57,33,0,8.66,clear sky
15,15,Vila Velha,BR,-20.3297,-40.2925,82.99,83,75,8.05,broken clouds
19,19,Mataura,NZ,-46.1927,168.8643,77.25,50,0,2.44,clear sky
23,23,Casa Nova,BR,-9.4167,-41.1333,86.00,35,0,4.61,light rain
27,27,Samarai,PG,-10.6167,150.6667,85.84,68,84,5.12,broken clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                190
City                   190
Country                190
Lat                    190
Lng                    190
Max Temp               190
Humidity               190
Cloudiness             190
Wind Speed             190
Current Description    190
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
Preferred_cities_df =preferred_cities_df.dropna()
#Preferred_cities_df.columns

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sampit,ID,76.46,light rain,-2.5333,112.9500,
1,Palauig,PH,80.65,scattered clouds,15.4350,119.9045,
5,Rikitea,PF,78.67,overcast clouds,-23.1203,-134.9692,
10,Itaituba,BR,86.00,clear sky,-4.2761,-55.9836,
13,Kahului,US,77.00,few clouds,20.8947,-156.4700,
14,Prieska,ZA,79.57,clear sky,-29.6641,22.7474,
15,Vila Velha,BR,82.99,broken clouds,-20.3297,-40.2925,
19,Mataura,NZ,77.25,clear sky,-46.1927,168.8643,
23,Casa Nova,BR,86.00,light rain,-9.4167,-41.1333,
27,Samarai,PG,85.84,broken clouds,-10.6167,150.6667,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df["Hotel Name"]= hotel_df["Hotel Name"].replace ("",np.nan)

clean_hotel_df= hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sampit,ID,76.46,light rain,-2.5333,112.9500,Aquarius Boutique Hotel Sampit
1,Palauig,PH,80.65,scattered clouds,15.4350,119.9045,Myhousebythesea
5,Rikitea,PF,78.67,overcast clouds,-23.1203,-134.9692,Pension Maro'i
10,Itaituba,BR,86.00,clear sky,-4.2761,-55.9836,Hotel Campos
13,Kahului,US,77.00,few clouds,20.8947,-156.4700,Maui Seaside Hotel
...,...,...,...,...,...,...,...
686,Caravelas,BR,77.74,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
690,Palu,ID,81.45,overcast clouds,-0.8917,119.8707,Hotel Santika Palu
692,Cockburn Town,TC,77.00,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
704,Morondava,MG,79.90,light rain,-20.2833,44.2833,La Case Bambou


In [22]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))